In [42]:
%load_ext autoreload
%autoreload 2
import numpy as np
from Enviroment import Dealer, BlackjackEnv
from agent import Agent

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

dealer = Dealer()
dealer.create_playing_deck(D = np.inf)
#print last 10 cards in the deck
print(dealer.playing_deck[-10:])
print('deal five cards:')
#deal 5 cards 
for i in range(5):
    print(dealer.deal_card())
print('new upcoming cards:')
print(dealer.playing_deck[-10:]) 


[('clubs', 11), ('spades', 2), ('spades', 12), ('hearts', 3), ('spades', 11), ('clubs', 2), ('hearts', 8), ('clubs', 8), ('clubs', 1), ('hearts', 9)]
deal five cards:
('hearts', 9)
('clubs', 1)
('clubs', 8)
('hearts', 8)
('clubs', 2)
new upcoming cards:
[('spades', 5), ('clubs', 7), ('spades', 5), ('clubs', 6), ('spades', 4), ('clubs', 11), ('spades', 2), ('spades', 12), ('hearts', 3), ('spades', 11)]


In [467]:
env = BlackjackEnv(D=1)

In [376]:
env.reset_hand()
env.reset_deck(D=10)
env.stick_or_hit(hit=True)  # initial deal

stuck = False

while env.player_hand_value < 21 and stuck == False:
    print('Current player hand:', env.player_hand)
    print('player hand value:', env.player_hand_value)
    input_action = input("Do you want to hit? (y/n): ")
    
    if input_action.lower() == 'y': 
        env.stick_or_hit(hit=True) 
    else:
        stuck = True

if env.player_hand_value > 21:
    print('Player busts! value:', env.player_hand_value)
    print('Player hand:', env.player_hand)
else:
    print('Player sticks with value:', env.player_hand_value)
    print('Player hand:', env.player_hand)

print(f'cards left in deck: {(env.dealer.playing_deck_length)}')


Current player hand: [('diamonds', 7)]
player hand value: 7


KeyboardInterrupt: Interrupted by user

Testing the agent 

In [487]:
agent = Agent(env)
#baseline policy: always hit when below 16
initial_policy = ['hit','hit','hit','hit','hit','hit','hit','hit','hit','hit','hit','hit','hit','hit','hit','stick','stick','stick','stick','stick','stick']
agent.set_policy(initial_policy)


print('cards left in deck:',agent.env.dealer.playing_deck_length)

print('value from the hand: ',agent.play_hand())
print('cards delt: ',agent.env.player_hand)

cards left in deck: 0
value from the hand:  0
cards delt:  [None]


Test the run episode function, in future will add the policy update functionality in 

In [522]:
agent.env.reset_deck(D=2)
episode_reward =agent.run_episode_reward()
print('reward after one episode: ',episode_reward)

state17
state11
state16
state21
state19
state9
state19
state12
state13
state19
state18
state8
state18
state4
state14
state17
state9
state18
state13
state22
state11
state14
state24
state21
state14
state18
state20
state14
state19
state8
state10
state16
state21
state14
state24
state18
state15
state25
state14
state24
state18
state13
state18
state14
state15
state25
state20
state16
state19
state19
state15
state18
state13
state14
state21
state14
state24
state15
state24
state16
state6
state10
state20
state12
state14
state24
state14
reward after one episode:  9520
